# day 9

https://adventofcode.com/2018/day/9

In [ ]:
import os

import eri.logging as logging

In [ ]:
FNAME = os.path.join('data', 'day9.txt')

LOGGER = logging.getLogger('day9')
logging.configure()

## part 1

### problem statement:

> You talk to the Elves while you wait for your navigation system to initialize. To pass the time, they introduce you to their favorite marble game.
> 
> The Elves play this game by taking turns arranging the marbles in a circle according to very particular rules. The marbles are numbered starting with 0 and increasing by 1 until every marble has a number.
> 
> First, the marble numbered 0 is placed in the circle. At this point, while it contains only a single marble, it is still a circle: the marble is both clockwise from itself and counter-clockwise from itself. This marble is designated the current marble.
> 
> Then, each Elf takes a turn placing the lowest-numbered remaining marble into the circle between the marbles that are 1 and 2 marbles clockwise of the current marble. (When the circle is large enough, this means that there is one marble between the marble that was just placed and the current marble.) The marble that was just placed then becomes the current marble.
> 
> However, if the marble that is about to be placed has a number which is a multiple of 23, something entirely different happens. First, the current player keeps the marble they would have placed, adding it to their score. In addition, the marble 7 marbles counter-clockwise from the current marble is removed from the circle and also added to the current player's score. The marble located immediately clockwise of the marble that was removed becomes the new current marble.
> 
> For example, suppose there are 9 players. After the marble with value 0 is placed in the middle, each player (shown in square brackets) takes a turn. The result of each of those turns would produce circles of marbles like this, where clockwise is to the right and the resulting current marble is in parentheses:
> 
>     [-] (0)
>     [1]  0 (1)
>     [2]  0 (2) 1 
>     [3]  0  2  1 (3)
>     [4]  0 (4) 2  1  3 
>     [5]  0  4  2 (5) 1  3 
>     [6]  0  4  2  5  1 (6) 3 
>     [7]  0  4  2  5  1  6  3 (7)
>     [8]  0 (8) 4  2  5  1  6  3  7 
>     [9]  0  8  4 (9) 2  5  1  6  3  7 
>     [1]  0  8  4  9  2(10) 5  1  6  3  7 
>     [2]  0  8  4  9  2 10  5(11) 1  6  3  7 
>     [3]  0  8  4  9  2 10  5 11  1(12) 6  3  7 
>     [4]  0  8  4  9  2 10  5 11  1 12  6(13) 3  7 
>     [5]  0  8  4  9  2 10  5 11  1 12  6 13  3(14) 7 
>     [6]  0  8  4  9  2 10  5 11  1 12  6 13  3 14  7(15)
>     [7]  0(16) 8  4  9  2 10  5 11  1 12  6 13  3 14  7 15 
>     [8]  0 16  8(17) 4  9  2 10  5 11  1 12  6 13  3 14  7 15 
>     [9]  0 16  8 17  4(18) 9  2 10  5 11  1 12  6 13  3 14  7 15 
>     [1]  0 16  8 17  4 18  9(19) 2 10  5 11  1 12  6 13  3 14  7 15 
>     [2]  0 16  8 17  4 18  9 19  2(20)10  5 11  1 12  6 13  3 14  7 15 
>     [3]  0 16  8 17  4 18  9 19  2 20 10(21) 5 11  1 12  6 13  3 14  7 15 
>     [4]  0 16  8 17  4 18  9 19  2 20 10 21  5(22)11  1 12  6 13  3 14  7 15 
>     [5]  0 16  8 17  4 18(19) 2 20 10 21  5 22 11  1 12  6 13  3 14  7 15 
>     [6]  0 16  8 17  4 18 19  2(24)20 10 21  5 22 11  1 12  6 13  3 14  7 15 
>     [7]  0 16  8 17  4 18 19  2 24 20(25)10 21  5 22 11  1 12  6 13  3 14  7 15
>
> The goal is to be the player with the highest score after the last marble is used up. Assuming the example above ends after the marble numbered 25, the winning score is 23+9=32 (because player 5 kept marble 23 and removed marble 9, while no other player got any points in this very short example game).
> 
> Here are a few more examples:
> 
> + 10 players; last marble is worth 1618 points: high score is 8317
> + 13 players; last marble is worth 7999 points: high score is 146373
> + 17 players; last marble is worth 1104 points: high score is 2764
> + 21 players; last marble is worth 6111 points: high score is 54718
> + 30 players; last marble is worth 5807 points: high score is 37305
>
> What is the winning Elf's score?

#### loading data

In [ ]:
test_data = [
    {'input': (9, 25), 'value': 32},
    {'input': (10, 1618), 'value': 8317},
    {'input': (13, 7999), 'value': 146373},
    {'input': (17, 1104), 'value': 2764},
    {'input': (21, 6111), 'value': 54718},
    {'input': (30, 5807), 'value': 37305},
]

In [ ]:
def load_data(fname=FNAME):
    # hard coding
    num_players = 410
    high_marble_val = 72059
    return num_players, high_marble_val

#### function def

In [ ]:
import tqdm

class Game(object):
    def __init__(self, num_players, high_marble_val, prog_freq=100):
        self.num_players = num_players
        self.high_marble_val = high_marble_val
        self.circle = [0]
        self.current_pos = 0
        self.current_marble_val = 1
        self.current_player = 0
        self.scores = {i: 0 for i in range(num_players)}
        self.over = False
        self.log_circle()
        self.prog_freq = prog_freq
        self.progbar = tqdm.tqdm_notebook(total=self.high_marble_val)
    
    def take_turn(self):
        if self.current_marble_val > self.high_marble_val:
            raise ValueError("how did we get a marble value higher than the highest allowed?")
        
        if self.current_marble_val % 23 == 0 and self.current_marble_val > 0:
            LOGGER.debug("special move!")
            LOGGER.debug(
                'marble {} for player {}'.format(self.current_marble_val, self.current_player)
            )
            # special 23 move: current player keeps current marble.
            # find marble 7ccw from current pos; player keeps that too
            # current pos becomes the position that had been 7ccw
            self.scores[self.current_player] += self.current_marble_val
            
            seven_ccw_ind = (self.current_pos - 7) % len(self.circle)
            self.scores[self.current_player] += self.circle.pop(seven_ccw_ind)
            self.current_pos = seven_ccw_ind
            self.log_circle()
        else:
            # regular move: current player puts the current marble
            # between the 1cw and 2cw (1 right and 2 right) marbles
            one_cw_ind = (self.current_pos + 1) % len(self.circle)
            
            self.circle.insert(one_cw_ind + 1, self.current_marble_val)
            
            # iterate the player and current marble value
            self.current_pos = self.circle.index(self.current_marble_val)
            self.log_circle()
            
        self.current_marble_val += 1
        self.current_player += 1
        self.current_player %= self.num_players

        # set game over if that makes sense
        self.over = self.current_marble_val > self.high_marble_val
        
        if (self.current_marble_val % self.prog_freq == 0) or (self.current_marble_val == self.high_marble_val):
            self.progbar.update(n=self.prog_freq)
            self.progbar.refresh()
        
    
    def winning_score(self):
        return max(self.scores.values())
    
    def log_circle(self):
        if len(self.circle) > 20:
            return
        circle_str = ''.join([
            ('({})'.format(val) if i == self.current_pos else ' {} '.format(val))
            for (i, val) in enumerate(self.circle)
        ])
        LOGGER.debug('circle: {}'.format(circle_str))

In [ ]:
def q_1(num_players, high_marble_val, prog_freq=100):
    g = Game(num_players, high_marble_val, prog_freq)
    while not g.over:
        g.take_turn()
    return g.winning_score()

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    for test in test_data:
        LOGGER.debug('new test: {}'.format(test))
        assert q_1(*test['input']) == test['value']
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(*load_data())

## part 2

### problem statement:

> Amused by the speed of your answer, the Elves are curious:
> 
> What would the new winning Elf's score be if the number of the last marble were 100 times larger?

#### function def

In [ ]:
def q_2(num_players, high_marble_val):
    return q_1(num_players, 100 * high_marble_val, prog_freq=10_000)

#### answer

In [ ]:
q_2(*load_data())

In [ ]:
from collections import deque, defaultdict

def play_game(max_players, last_marble):
    scores = defaultdict(int)
    circle = deque([0])

    for marble in range(1, last_marble + 1):
        if marble % 23 == 0:
            circle.rotate(7)
            scores[marble % max_players] += marble + circle.pop()
            circle.rotate(-1)
        else:
            circle.rotate(-1)
            circle.append(marble)

    return max(scores.values()) if scores else 0

In [ ]:
play_game(*load_data())

In [ ]:
num_players, high_marble_val = load_data()
play_game(num_players, 100 * high_marble_val)

fin